In [2]:
import os
import re

from importlib import reload

# os.environ['OPENAI_API_KEY'] = ""

In [9]:
import easy_agents.agents as agents
import easy_agents.llm.openai as openai_llm
import wikipediaapi

reload(agents)

system_prompt = """
You are an AI assistant that follows the React prompting technique. Your process 
involves a loop of Thought, Action, and Observation, concluding with an Answer.

Here's how you should operate:

1. Thought: Describe your reasoning about the given question or task.
2. Action: Choose and execute one of the available actions.
3. Observation: You will receive the result of the action you chose.
4. Repeat steps 1-3 as necessary until you can provide a final answer.
5. Answer: Provide the final response to the original question or task.

Available actions:
1. calculate: Perform mathematical calculations
   Usage: calculate: <mathematical expression>
   Example: calculate: 2+346-677*78/6+sqrt((10 + 2))+7

2. wikipedia: Search Wikipedia for information
   Usage: wikipedia: <search term>
   Example: wikipedia: Barack Obama
   Important notes:
   - The input to this action must be a noun.
   - The output from this action is a summary of the search result from Wikipedia.
   - You cannot search for multiple pages or nouns at once.

Always format your responses as follows:
Thought: <your reasoning>
Action: <chosen action>: <action input>

After receiving an observation, continue with:
Thought: <your reasoning based on the new information>
Action: <next chosen action>: <action input>

When you're ready to give a final answer:
Answer: <your final response>

Remember to use the available actions wisely and think through each step of your process.
""".strip()

actions_re = re.compile("^Action: (\w+): (.*)")


def wikipedia(q):
    wiki_wiki = wikipediaapi.Wikipedia('MyProjectName', 'en')
    summary = wiki_wiki.page(q).summary

    return summary


def calculate(query: str):
    return eval(query)


wikipedia_tool = agents.Tool(
    name="wikipedia", description="Search Wikipedia", function=wikipedia
)

calculate_tool = agents.Tool(
    name="calculate", description="Run a calculation", function=calculate
)

llm = openai_llm.OpenAI(model="gpt-4o-mini")

my_agent = agents.Agent(llm=llm, max_turns=10, system_prompt=system_prompt)

my_agent.add_tool(wikipedia_tool)
my_agent.add_tool(calculate_tool)

res = my_agent.run("What is the square root of the height of the Empire State Building divided by the height of the Golden Gate Bridge?")

Thought: To answer this question, I need to find the heights of both the Empire State Building and the Golden Gate Bridge. Once I have these values, I can divide the height of the Empire State Building by the height of the Golden Gate Bridge and then take the square root of the result.

Action: wikipedia: Empire State Building
----------
Tool call:
	Name: wikipedia
	Input: Empire State Building
	Output: The Empire State Building is a 102-story Art Deco skyscraper in the Midtown South neighborhood of Manhattan in New York City. The building was designed by Shreve, Lamb & Harmon and built from 1930 to 1931. Its name is derived from "Empire State", the nickname of the state of New York. The building has a roof height of 1,250 feet (380 m) and stands a total of 1,454 feet (443.2 m) tall, including its antenna. The Empire State Building was the world's tallest building until the first tower of the World Trade Center was topped out in 1970; following the September 11 attacks in 2001, the Emp

NameError: name 'sqrt' is not defined